In [1]:
import pandas as pd

In [4]:
countries = ['CZE', 'POL', 'HUN']

In [5]:
#give function df to cut, the column were you want to define a slice and a list of values which should be sliced
#get a list of dfs, one for each defined value of cut-list, NaNs are dropped, index is datetime
def cut_df(df, column_to_cut, cut):
    covid_dfs = []
    for i in range(len(cut)):
        cut_df = df.where(df[column_to_cut] == cut[i]).dropna(how='all')
        cut_df = cut_df.dropna(axis=1, how='all')
        cut_df.index = pd.to_datetime(cut_df.index)
        covid_dfs.append(cut_df)
    return pd.Series(data = covid_dfs, index = cut)

In [2]:
exchange_df = pd.read_parquet('data/FRED_Req.parquet')
covid_df = pd.read_parquet('data/OWID_Req.parquet')
nasdaq_df = pd.read_parquet('data/QUANDL_Req.parquet')

In [8]:
#make df from FRED data, convert index to datetime
exchange_df.index = pd.to_datetime(exchange_df.index)
exchange_df = exchange_df.loc['2020-03-01':,:].astype(float) #this does work but with warning

In [10]:
#convert nasdaq index to datetime, reverse df
nasdaq_df.index = pd.to_datetime(nasdaq_df.index)
nasdaq_df = nasdaq_df.iloc[::-1]

,CZ,PL,HU
Trade Date,,,
2020-01-31,705.91,773.37,1491.67
2020-02-03,711.52,774.20,1506.36
2020-02-04,714.82,789.20,1525.22
2020-02-05,721.66,798.54,1529.18
2020-02-06,725.67,790.92,1521.56
...,...,...,...
2021-07-22,797.99,862.04,1633.68
2021-07-23,791.77,863.07,1609.93
2021-07-26,794.52,862.26,1588.86


In [9]:
covid_dfs = cut_df(covid_df, 'iso_code', countries)

In [10]:
#problem: FRED data monthly, OWID data daily -> need to convert OWID data
#add up cases of a month, save in a list
list_results = []
for j in range(len(covid_dfs)):
    list_monthly_cases = []
    list_monthly_cases_index = []
    monthly_cases = covid_dfs[j].iloc[0]['new_cases']
    for i in range(len(covid_dfs[j])-1):
        if covid_dfs[j].index[i].month == covid_dfs[j].index[i+1].month:
            monthly_cases = monthly_cases + covid_dfs[j].iloc[i+1]['new_cases']
        else:
            list_monthly_cases.append(monthly_cases)
            list_monthly_cases_index.append(covid_dfs[j].index[i+1])
            monthly_cases = covid_dfs[j].iloc[i+1]['new_cases']
    #safe list of monthly cases as df, concat with df on CZK/USD data
    monthly_cases_summed = pd.DataFrame(data = list_monthly_cases, index = list_monthly_cases_index)
    if j == 0:
        monthly_cases_summed_df = monthly_cases_summed
    else:
        monthly_cases_summed_df[j] = monthly_cases_summed
monthly_cases_summed_df.columns = countries    

In [11]:
#safe dfs to parquet
monthly_cases_summed_df.to_parquet('data/monthly_cases_summed.parquet')
exchange_df.to_parquet('data/exchange.parquet')

In [2]:
#combine dfs with multiindex
'''
can't save multiindex to parquet so I saved both dfs and combine them in data analyser

results = pd.concat([exchange_df, monthly_cases_summed_df], keys=['local currency to USD', 'summed monthly COVID cases'], axis='columns')
'''

"\ncan't save multiindex to parquet so I saved both dfs and combine them in data analyser\n\nresults = pd.concat([exchange_df, monthly_cases_summed_df], keys=['local currency to USD', 'summed monthly COVID cases'], axis='columns')\n"

In [10]:
'''
#control if idea works -> it does
print(list_monthly_cases)
print(covid_df.loc['2020-04-30']['total_cases']-covid_df.loc['2020-03-31']['total_cases'])
covid_df.loc['2020-03-31']['total_cases']
'''

"\n#control if idea works -> it does\nprint(list_monthly_cases)\nprint(covid_df.loc['2020-04-30']['total_cases']-covid_df.loc['2020-03-31']['total_cases'])\ncovid_df.loc['2020-03-31']['total_cases']\n"

In [1]:
#results.to_parquet('data/merged_data.parquet')